In [1]:
# Python 3.X
import numpy as np
from pandas import Series, DataFrame
from pandas.tseries.offsets import MonthEnd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp
np.set_printoptions(precision=3)
import datetime
plt.rc('figure', figsize=(10, 8))
from numpy.linalg import inv
from numpy import dot
import re

# FINM 8006 Assignment 

* Due COB 14 Oct, 2022
* Submit one copy each group with all student name and UID clearly typed. 
* You have the option of completing your assignment directly in this notebook (just insert cells in the .ipynb file). You should have Jupyter notebook installed if you want to use this. 
* You can alternatively write your answers in the editor of your choosing (Words, etc.), and run your code in a separate file. If you use a separate programming file, please submit both your answer file and the programming file. 
* Innitial data loading and cleaning is given in the steps below. 

## Mean Variance Optimization

Data of monthly returns is given in asgn1.csv, where date is the end of the return months, RET is the montly return, PERMNO is the stock code, rf is the risk free rate.

### Q1.1

Suppose you are about to invest at the beginning of January 2005, and you want to acheive mean-variance efficiency using these 5 stocks. You estimate of the mean and variance of the risky stocks using data from the past 36 months.  The risk free rate is known at the beginning of the periods.

If your desired rate of return is 3% in the month, find the **optimal weights on the stocks, and the expected mean and standard deviation of your portfolio.** 

In [65]:
def load_mv_data():
    stocks=DataFrame([{'Apple':14593, 'IBM':12490, 'Microsoft':10107, 'Nike':57665, 'Disney':26403}], 
                 index=['PERMNO']).T
    stocks.index.name = 'stocks'
    stocks = stocks.reset_index().sort_values(['PERMNO'])
    msf = pd.read_csv('asgn1.csv')
    msf['date']=pd.to_datetime(msf['date'])
    msf = msf.merge(stocks, on='PERMNO')
    msf = msf.sort_values(['PERMNO', 'date'])
    return msf, stocks

In [66]:
msf, stocks = load_mv_data()
msf.head(3)

,date,PERMNO,RET,rf,stocks
0,1994-01-31,10107,0.055814,0.0025,Microsoft
1,1994-02-28,10107,-0.030837,0.0021,Microsoft
2,1994-03-31,10107,0.027273,0.0027,Microsoft


### Q1.2
Plot the mean-variance efficeint curve of these assets when you make your investment decision. 

## Asset Pricing in Chinese Markets

Many have commented that the Chinese stock market is very different from the more mature markets. In this assignment, you will explore some aspects of the asset pricing in Chinese markets.

#### Loading data

You are given three data sets from year 2000: 'csmar_be.csv' has the December book value (BE) of firms (STKCD). 'csmarstk0016.csv' has monthly market value (ME) and return (ret) information of individual stocks. Data is cleaned to exclude micro stoks and penny stocks. 'csmarmktretrf.csv' has market and risk free returns. Market return here is proxied by returns on the SSE. 

* ME: market value of equity
* STKCD: stock code
* MARKETTYPE: market type, 1 is Shanghai, 4 is Shengzhen
* BE: book value of equity
* ret: stock return
* mktret: market return
* rf: risk free rate

In [5]:
def load_stock_returns():
    # markettype 1 Shanghai A, 4, Shengzhen A
    df=pd.read_csv('csmarstk0016.csv')
    df['TRDMNT'] = pd.to_datetime(df['TRDMNT'].astype('str'), format='%Y-%m-%d')
    df = df.rename(columns={'TRDMNT':'date'})
    df = df.loc[df['ret'].notnull()]
    return df

In [6]:
stkret = load_stock_returns()
stkret.head()

,STKCD,MARKETTYPE,date,ME,ret
0,1,4,2000-01-31,2.875573e+10,0.061891
1,1,4,2000-02-29,2.842984e+10,-0.011333
2,1,4,2000-03-31,2.850743e+10,0.002729
3,1,4,2000-04-30,2.956269e+10,0.037017
4,1,4,2000-05-31,2.793325e+10,-0.055118


In [15]:
def load_market_return():
    df = pd.read_csv('csmarmktretrf.csv')
    df['TRDMNT'] = pd.to_datetime(df['TRDMNT'].astype('str'), format='%Y-%m-%d')
    df = df.rename(columns={'TRDMNT':'date'})
    df = df.drop('mktretxd', axis=1)
    return df

In [16]:
mktret = load_market_return()
mktret.head(3)

,date,mktret,rf
0,2000-01-31,0.123413,0.001856
1,2000-02-29,0.118776,0.001856
2,2000-03-31,0.049308,0.001856


In [17]:
def load_book_value():
    # December Book Equity where BE >0
    df= pd.read_csv('csmar_be.csv')
    df['date'] = pd.to_datetime(df['date'].astype('str'), format='%Y-%m-%d') 
    df = df.loc[df['BE']>0]
    df = df.sort_values(["STKCD", 'date', 'BE'])
    df = df.groupby(['STKCD', 'date']).last().reset_index()
    return df

In [86]:
decbe = load_book_value()
decbe.head()

,STKCD,date,BE
0,1,1999-12-31,2.900831e+09
1,1,2000-12-31,4.738884e+09
2,1,2001-12-31,3.627669e+09
3,1,2002-12-31,3.768021e+09
4,1,2003-12-31,3.965084e+09


### Q2.1 
Assume that power utility and relative risk aversion is 3, and constent expected market excess return.
* What is the optimal proportion of wealth to invest in the risky (stock) assets? (you will invest in a market portfolio). 
* Your answer above may be different from the typical answer in the U.S. market, explain what mainly drives this difference?

### Size and Value

#### Forming Portfolios

According to size and MTB.

Starting from '2000-7-31', form a 5X5 portfolios of size and book-to-market. Size is defined by end of June market value of equity, value is defined by December book value of equity in the previous year. Portfolio is rebalanced at each July. 
* You may want to create a 'year' variable that designate portfolio formation year.

In [81]:
# year variable here is the portfolio formation year
stkret['year'] = np.where(stkret['date'].dt.month>=7, stkret['date'].dt.year, stkret['date'].dt.year-1)

### Q2.2 
Show the CAPM alpha and beta of these portfolios in respective tables

### Q2.3
Calculate actual mean portfolio returns and compare them with the CAPM predicted average portfolio returns, explain any patterns in the data. 

### Q2.4 
Plot the mean excess return against CAPM predicted return, plot both within size (BM) plots and the within BM (size) plots. In the U.S. data, the most severe violation of CAPM happens along BTM portfolios, What about China? Can you explain why this factor comes up so significant in the Chinese markets?

### Constructing SMB and HML factors

Construct 2x3 portfolios of size and BTM according to the Shanghai Stock Exchange cutoff (MARKETTYPE==1). BTM cutoff is (0.3,0.7), Size cutoff is 0.5. Calculate the value-weighted returns of the portfolios and then construct SMB and HML factors following Fama French (93) procedure. 

### Q2.5
Plot the cumulative returns of the SMB and HML factors for the Chinese marlet. 

### Q2.6 
Tabulate the coefficient estimates of the Fama-French 3 factor model of the 25 (5x5) sorted portfolio returns. Does the coefficients estimates make sense? 

### Q2.7 
plot the Mean portfolio return against the Fama-French 3 factor predicted return again for both Within-BM Size Plot and Within-Size BM Plot. How do the graphs compare to the earlier CAPM plots? 